# --------------------------------
# Instalação e importação de Dependências
# --------------------------------

In [3]:
import keras
import os
import gdown
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense


from keras import layers
from keras import backend as K
from keras import regularizers
from keras.callbacks import TensorBoard
from ActVibModules.ActVibSystem import ActVibData 

In [ ]:
os.getcwd()

MUDAR QUANDO TIVER DATASET

In [ ]:
# Lista de tuplas contendo o ID do arquivo e o URL de cada arquivo
arquivos_para_baixar = [
    ('MedicaoDEDOjoao', 'https://drive.google.com/file/d/1waQTn6e9-QB5OPWqSTHl4HRZRisL7-rv/view?usp=drive_link'),
    ('MedicaoMETAL2joao', 'https://drive.google.com/file/d/19XDZhFSeum3GM6W2bq2fSOLUs0C-DO5d/view?usp=drive_link'),
    ('MedicaoMetalJOAO', 'https://drive.google.com/file/d/1B-3I_9Dd6cGZ2ccoiEbdP8B-pa4gZZKy/view?usp=drive_link'),
    ('MedicoesDedo2joao', 'https://drive.google.com/file/d/1HV1VR9SBKw7eI1yAJ33VNADVXSZVe_mV/view?usp=drive_link'),
    ('Dados_Impacto_Amortecido.feather', 'https://drive.google.com/file/d/1OlG9XfdQwxlVRHushhaP-SRYaKKO_D0n/view?usp=drive_link'),
    ('Dados_Impacto_Metalico.feather', 'https://drive.google.com/file/d/1mFfjhsMNsKoNTp-kmEL6gcK_tzwh8StB/view?usp=drive_link')
]

# Itera sobre cada tupla e baixa o arquivo correspondente
for arquivo_id, arquivo_url in arquivos_para_baixar:
    # Baixando o arquivo
    gdown.download(arquivo_url, quiet=False, fuzzy=True)

In [ ]:
dadosDedo = ActVibData("MedicaoDEDOjoao.feather")
dadosMetal = ActVibData("MedicaoMetalJOAO.feather")
dadosDedo2 = ActVibData("MedicoesDedo2joao.feather")
dadosMetal2 = ActVibData("MedicaoMETAL2joao.feather")
dadosImpactoAmortecido = ActVibData("Dados_Impacto_Amortecido.feather")
dadosImpactoMetalico = ActVibData("Dados_Impacto_Metalico.feather")

# -----------------------
# 1 - Pré-normalização
# -----------------------

In [ ]:
scaler = StandardScaler()
normal_data_scaled = scaler.fit_transform(normal_data)
clamped_data_scaled = scaler.transform(clamped_data)

# -------------------------------------
# 2 - Construir modelo do autoencoder
# -------------------------------------

In [ ]:
def build_autoencoder(input_dim):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(64, activation='relu')(input_layer)
    encoded = Dense(32, activation='relu')(encoded)
    encoded = Dense(16, activation='relu')(encoded)

    decoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(64, activation='relu')(decoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)

    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

# -------------------------------------
# 3 - Treinando o modelo (dataset - viga normal)
# -------------------------------------

In [ ]:
input_dim = normal_data_scaled.shape[1]
autoencoder_normal = build_autoencoder(input_dim)
autoencoder_normal.fit(normal_data_scaled, normal_data_scaled,
                       epochs=50, batch_size=32, shuffle=True, validation_split=0.2)

# -------------------------------------
# 4 - Visualizar os resultados (viga normal)
# -------------------------------------

In [ ]:
# Vamos reconstruir os dados e calcular o erro de reconstrução para identificar anomalias
reconstructed_normal = autoencoder_normal.predict(normal_data_scaled)
reconstruction_error_normal = np.mean(np.abs(reconstructed_normal - normal_data_scaled), axis=1)

In [ ]:
# Visualizar o erro de reconstrução
plt.figure(figsize=(10, 6))
plt.hist(reconstruction_error_normal, bins=50)
plt.xlabel('Reconstruction Error')
plt.ylabel('Frequency')
plt.title('Histogram of Reconstruction Error (Normal Beam)')
plt.show()

# -------------------------------------
# 3 - Treinando o modelo (dataset - viga engastada)
# -------------------------------------

In [ ]:
input_dim = clamped_data_scaled.shape[1]
autoencoder_clamped = build_autoencoder(input_dim)
autoencoder_clamped.fit(clamped_data_scaled, clamped_data_scaled,
                       epochs=50, batch_size=32, shuffle=True, validation_split=0.2)

# -------------------------------------
# 4 - Visualizar os resultados (viga engastada)
# -------------------------------------

In [ ]:
# Vamos reconstruir os dados e calcular o erro de reconstrução para identificar anomalias
reconstructed_clamped = autoencoder_clamped.predict(clamped_data_scaled)
reconstruction_error_clamped = np.mean(np.abs(reconstructed_clamped - clamped_data_scaled), axis=1)

In [ ]:
# Visualizar o erro de reconstrução
plt.figure(figsize=(10, 6))
plt.hist(reconstruction_error_clamped, bins=50)
plt.xlabel('Reconstruction Error')
plt.ylabel('Frequency')
plt.title('Histogram of Reconstruction Error (Clamped Beam)')
plt.show()